In [15]:
import country_converter as coco
import requests
import geopandas as gpd
import pandas as pd

In [16]:
from shapely.geometry import Point
from tqdm.notebook import tqdm

## Get DoDo Data

In [17]:
url = "https://globalapi.dodopizza.com/api/v1/pizzerias/count"
r = requests.get(url)

In [18]:
dodo_df = pd.DataFrame(r.json()["countries"])
dodo_df = dodo_df.rename(
    columns={"countryCode": "iso_3166_1_alpha_2_codes", "pizzeriaCount": "DODOs"}
)
dodo_df["iso_3166_1_alpha_2_codes"] = dodo_df["iso_3166_1_alpha_2_codes"].str.upper()

In [19]:
dodo_df = dodo_df[["iso_3166_1_alpha_2_codes", "DODOs"]]

## Get DoDo Locations

In [20]:
countries = r.json()["countries"]

In [21]:
pizzeria_list = []
for country in countries:
    r = requests.get(
        f"https://globalapi.dodopizza.com/api/v1/pizzerias/all/{country['countryId']}"
    )
    pizzerias = r.json()["countries"][0]["pizzerias"]
    for pizzeria in pizzerias:
        pizzeria_list.append(
            Point(pizzeria["coords"]["long"], pizzeria["coords"]["lat"])
        )

In [25]:
dodo_pizzeria_locations_gdf = gpd.GeoDataFrame({"geometry": pizzeria_list}, crs=4326)

In [27]:
dodo_pizzeria_locations_gdf.to_file("data/dodo_locations_v2.gpkg")

## Open World data

In [73]:
country_pop_df = pd.read_csv(
    "data/API_SP.POP.TOTL_DS2_en_csv_v2_87/API_SP.POP.TOTL_DS2_en_csv_v2_87.csv"
)

In [74]:
country_pop_df["iso_3166_1_alpha_2_codes"] = country_pop_df.apply(
    lambda row: coco.convert(names=[row["Country Code"]], to="ISO2"), axis=1
)

AFE not found in ISO3
AFW not found in ISO3
ARB not found in ISO3
CEB not found in ISO3
CHI not found in ISO3
CSS not found in ISO3
EAP not found in ISO3
EAR not found in ISO3
EAS not found in ISO3
ECA not found in ISO3
ECS not found in ISO3
EMU not found in ISO3
EUU not found in ISO3
FCS not found in ISO3
HIC not found in ISO3
HPC not found in ISO3
IBD not found in ISO3
IBT not found in ISO3
IDA not found in ISO3
IDB not found in ISO3
IDX not found in ISO3
INX not found in ISO3
LAC not found in ISO3
LCN not found in ISO3
LDC not found in ISO3
LIC not found in ISO3
LMC not found in ISO3
LMY not found in ISO3
LTE not found in ISO3
MEA not found in ISO3
MIC not found in ISO3
MNA not found in ISO3
NAC not found in ISO3
OED not found in ISO3
OSS not found in ISO3
PRE not found in ISO3
PSS not found in ISO3
PST not found in ISO3
SAS not found in ISO3
SSA not found in ISO3
SSF not found in ISO3
SST not found in ISO3
TEA not found in ISO3
TEC not found in ISO3
TLA not found in ISO3
TMN not fo

In [75]:
country_pop_df = country_pop_df[["iso_3166_1_alpha_2_codes", "2023"]]

In [76]:
world_boundaries = gpd.read_file("data/world_boundaries.gpkg")

In [77]:
world_boundaries_reproj = world_boundaries.to_crs(3576)
world_boundaries_reproj.to_file("data/world_boundaries_reproj.gpkg")

## Merge data

In [78]:
dodo_counts_df = dodo_df.merge(
    world_boundaries_reproj[["geometry", "iso_3166_1_alpha_2_codes"]],
    on="iso_3166_1_alpha_2_codes",
    how="left",
)

In [80]:
dodo_counts_population_gdf = dodo_counts_df.merge(
    country_pop_df, on="iso_3166_1_alpha_2_codes", how="left"
)

In [81]:
dodo_counts_population_gdf = dodo_counts_population_gdf.rename(
    columns={"iso_3166_1_alpha_2_codes": "COUNTRY"}
)
dodo_counts_population_gdf = dodo_counts_population_gdf[
    ["DODOs", "COUNTRY", "geometry", "2023"]
]

In [82]:
dodo_counts_population_gdf["per_100k"] = (
    dodo_counts_population_gdf["DODOs"] / (dodo_counts_population_gdf["2023"] / 100_000)
).round(decimals=2)
dodo_counts_population_gdf["per_1m"] = (
    dodo_counts_population_gdf["DODOs"]
    / (dodo_counts_population_gdf["2023"] / 1_000_000)
).round(decimals=2)

In [83]:
dodo_counts_population_gdf.sort_values("per_1m", ascending=False)

,DODOs,COUNTRY,geometry,2023,per_100k,per_1m
4,985,RU,"MULTIPOLYGON (((3403040.93 -3796309.94, 340002...",143826130.0,0.68,6.85
7,48,BY,"MULTIPOLYGON (((-3527768.256 -2185935.142, -35...",9178298.0,0.52,5.23
2,104,KZ,"MULTIPOLYGON (((-206621.28 -4461148.288, -2090...",20330104.0,0.51,5.12
0,5,EE,"MULTIPOLYGON (((-3185560.803 -1351196.362, -31...",1370286.0,0.36,3.65
16,3,CY,"MULTIPOLYGON (((-4847973.754 -3284793.567, -48...",1344976.0,0.22,2.23
3,6,LT,"MULTIPOLYGON (((-3561985.219 -1366234.642, -35...",2871585.0,0.21,2.09
1,12,KG,"MULTIPOLYGON (((-877920.364 -5100677.016, -877...",7099750.0,0.17,1.69
8,3,SI,"MULTIPOLYGON (((-4539707.301 -1354024.532, -45...",2120461.0,0.14,1.41
14,4,AM,"MULTIPOLYGON (((-3790449.151 -3999941.976, -37...",2990900.0,0.13,1.34
13,10,AE,"MULTIPOLYGON (((-4077961.819 -5605599.919, -40...",10483751.0,0.10,0.95


In [84]:
dodo_counts_gdf = gpd.GeoDataFrame(dodo_counts_df, crs=3576)
dodo_counts_gdf.to_file("data/dodo_country_counts.gpkg")